In [0]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import time

# Flowster forum home page
baseURL = 'https://forum.flowster.app'

# Open Chrome web client using Selenium and retrieve page source
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(executable_path = "./chromedriver", options=options)

# Get base HTML text and generate soup object
driver.get(baseURL)
baseHTML = driver.page_source
baseSoup = BeautifulSoup(baseHTML, 'html.parser')

# Find all anchor objects that contain category information
categoryAnchors = baseSoup.find_all('a', class_='category-title-link')

# Get hyperlink reference and append it to the base URL to get the category page URL
categoryPageURL = []
for i in range(12):
    href = categoryAnchors[i]['href']
    categoryPageURL.append(baseURL + href)

#1st for_loop ro run through all categories
for c in categoryPageURL:
    # Get category HTML text
    driver.get(c)

    # Load the entire webage by scrolling
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while (True):
        # Scroll to bottom of page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for new page segment to load
        time.sleep(0.5)

        # Calculate new scroll height and compare with last scroll height
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            break
        lastHeight = newHeight
        
    # Generate soup object
    categoryHTML = driver.page_source
    categorySoup = BeautifulSoup(categoryHTML, 'html.parser')

    # Find all anchor objects that contain topic information
    topicAnchors = categorySoup.find_all('a', class_='title raw-link raw-topic-link')
    
    # Get hyperlink reference and append it to the base URL to get the topic page URL
    topicPageURL = []
    
    #2nd for_loop to run through all topics
    for j in range(len(topicAnchors)):
        href = topicAnchors[j]['href']
        topicPageURL.append(baseURL + href)
    
    #####Functions:#####
    def get_leadc(topic_url):
        page = requests.get(topic_url)
        topicSoup = BeautifulSoup(page.text, 'html.parser') 
        for c in topicSoup.find_all('meta', property='og:description'):
            leadc = c.get('content')
            return leadc
        
    def get_otherc(topic_url):
        driver.get(topic_url)
        topicHTML = driver.page_source
        topicSoup = BeautifulSoup(topicHTML, 'html.parser')
        postStream = topicSoup.find('div', class_='post-stream')
        postDivs = postStream.find_all('div', {'class':['topic-post clearfix regular','topic-post clearfix topic-owner regular']})
        otherc = []
        for i in range(1, len(postDivs)):
            post = postDivs[i].find('div', class_='cooked').text
            otherc.append(post)
        return otherc
        
    def get_tags(topic_url):
        page = requests.get(topic_url)
        topicSoup = BeautifulSoup(page.text, 'html.parser') 
        temp_tag = []
        for n in topicSoup.find_all('span', class_="category-name"):
            temp_tag.append(n.getText())
        return temp_tag
    
    def get_title(topic_url):
        page = requests.get(topic_url)
        topicSoup = BeautifulSoup(page.text, 'html.parser') 
        for i in topicSoup.find_all("meta", property="og:title"):
            topic = i.get("content")
            return topic
        
    def get_author(topic_url):
        page = requests.get(topic_url)
        topicSoup = BeautifulSoup(page.text, 'html.parser') 
        names = topicSoup.find_all("span", class_="creator")
        author = names[0].getText()
        return author
    
    def get_views(topic_url):
        driver.get(topic_url)
        topicHTML = driver.page_source
        topicSoup = BeautifulSoup(topicHTML, 'html.parser')
        views = topicSoup.find('li', class_='secondary views')
        if views == None:
            return 0
        return views.span.text
        
    def get_likes(topic_url):
        driver.get(topic_url)
        topicHTML = driver.page_source
        topicSoup = BeautifulSoup(topicHTML, 'html.parser')
        likes = topicSoup.find('li', class_='secondary likes')
        if likes == None:
            return 0
        return likes.span.text
    ###### end ######
    
    for t in topicPageURL:
    #Run the functions in this for loop 
        print(get_otherc(t))
        
    